# Знакомство с word2vec


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 37.8 gigabytes of available RAM

You are using a high-RAM runtime!


## Загрузка модели
Скачаем модель <code>google-news-vectors</code>. Откроем ее с помощью библиотеки <code>gensim</code>.

In [ ]:
! pip install --upgrade --no-cache-dir gdown
! pip install -q -U gensim
! gdown --id 0B7XkCwpI5KDYNlNUTTlSS21pQmM
! pip install -q SciPy==1.5.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
From (redirected): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t&uuid=2c370937-095d-4842-91fe-9a75caafac35
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:10<00:00, 156MB/s]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 67.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See

In [ ]:
! gunzip GoogleNews-vectors-negative300.bin.gz

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import gensim
from gensim.models import KeyedVectors

w = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin",
                                      binary=True)

Структура называется <code>KeyedVectors</code> и по сути представляет собой отображение между ключами и векторами. Каждый вектор идентифицируется своим ключом поиска, чаще всего коротким строковым токеном, поэтому обычно это соответствие между

<center><code>{str => 1D numpy array}</code></center><br/>



Например, выведем первые 10 координат вектора, соответствующего слову <code>sunrise</code>

In [ ]:
print("Размерность вектора: ", w["sunrise"].shape)
print("Первые 10 координат вектора: \n", w["sunrise"][:10])

Размерность вектора:  (300,)
Первые 10 координат вектора: 
 [-0.22558594 -0.03540039 -0.21679688  0.03613281 -0.2265625  -0.09814453
  0.109375   -0.34570312  0.18652344  0.01806641]


## Задание 1. Сходство.

Извлеките векторы слов <code>London</code>, <code>England</code>, <code>Moscow</code>. Посчитайте косинусное расстояние между словами <code>London</code> и <code>England</code> и между словами <code>Moscow</code> и <code>England</code>. Какая пара слов ближе? Подсказка: для вычисления косинусного расстояния использвется метод <code>distance()</code>. Правильный ответ представлен в блоке вывода.

In [ ]:
from scipy.spatial import distance

vec_london = w["London"]
vec_england = w["England"]
vec_moscow = w["Moscow"]

print("Cosine distance between London and England: ", float(distance.cosine(vec_london, vec_england)))
print("Cosine distance between Moscow and England: ", float(distance.cosine(vec_moscow, vec_england)))

Cosine distance between London and England:  0.5600714087486267
Cosine distance between Moscow and England:  0.8476868271827698


## Задание 2. Аналогии.
С помощью метода most_similar решите аналогию
```London : England = Moscow : X```

Правильный ответ представлен в блоке вывода.

(Подсказка: нужно использовать аргументы positive и negative)

In [ ]:
w.most_similar(positive=[vec_moscow, "Russia"])

[('Moscow', 0.98638916015625),
 ('Kremlin', 0.7748246192932129),
 ('Russian', 0.7569824457168579),
 ('Kiev', 0.7495608329772949),
 ('Tbilisi', 0.7209931015968323),
 ('Minsk', 0.7206584215164185),
 ('Ukraine', 0.71128249168396),
 ('Kyiv', 0.7099266648292542),
 ('St_Petersburg', 0.6833096742630005),
 ('Tashkent', 0.6815599203109741)]


## Задание 3. Сходство: найти лишнее.
С помощью метода <code>doesnt_match</code> найдите лишнее слово в ряду <code>breakfast cereal dinner lunch</code>.

Правильный ответ представлен в блоке вывода.

In [ ]:
str = "breakfast cereal dinner lunch"

extra_words = w.doesnt_match("breakfast cereal dinner lunch".split())

print("Лишнее слово: ", extra_words)

Лишнее слово:  cereal


## Задание 4. Представление предложений в виде векторов


Дано предложение: <code>the quick brown fox jumps over the lazy dog</code>. Вам нужно представить это предложение в виде вектора. Для этого найдите вектор каждого слова в модели, а затем усредните векторы покомпонентно.


In [ ]:
import numpy as np

# Define the sentence
sentence = "the quick brown fox jumps over the lazy dog"

# Split the sentence into words
words = sentence.split()

# Get the vector for each word in the sentence
vectors = [w[word] for word in words]

# Average the vectors component by component
sentence_vector = np.average(vectors, axis=0)

# Print the first 5 coordinates of the sentence vector
print(sentence_vector[:5])


[ 0.09055582  0.05434163 -0.06713867  0.10968696 -0.01060655]


## Загрузка ещё одной модели


Откроем модель google-news-vectors и модель, обученную на британском национальном корпусе http://vectors.nlpl.eu/repository/20/0.zip, с помощью gensim.


In [ ]:
! wget -c http://vectors.nlpl.eu/repository/20/0.zip
! unzip 0.zip
! head -3 model.txt

--2023-06-11 17:51:46--  http://vectors.nlpl.eu/repository/20/0.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344050746 (328M) [application/zip]
Saving to: ‘0.zip’

0.zip               100%[===================>] 328.11M  22.9MB/s    in 15s     

2023-06-11 17:52:02 (21.6 MB/s) - ‘0.zip’ saved [344050746/344050746]

Archive:  0.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  
163473 300
say_VERB -0.008861 0.097097 0.100236 0.070044 -0.079279 0.000923 -0.012829 0.064301 -0.029405 -0.009858 -0.017753 0.063115 0.033623 0.019805 0.052704 -0.100458 0.089387 -0.040792 -0.088936 0.110212 -0.044749 0.077675 -0.017062 -0.063745 -0.009502 -0.079371 0.066952 -0.070209 0.063761 -0.038194 -0.046252 0.049983 -0.094985 -0.086341 0.024665 -0.

Загрузим модель, обученную на британском национальном корпусе

In [ ]:
w_british = KeyedVectors.load_word2vec_format("model.bin", binary=True)

In [ ]:
try:
    print(w_british["London_NOUN"].shape)
    print('upper is ok')
except:
    print(w_british["london_NOUN"].shape)
    print('lower is ok')

(300,)
lower is ok


## Набор данных для оценки качества
Скачаем датасет wordsim353.



In [ ]:
! wget -c http://alfonseca.org/pubs/ws353simrel.tar.gz
! tar -xvf ws353simrel.tar.gz
! head -5 wordsim353_sim_rel/wordsim_similarity_goldstandard.txt

--2023-06-11 17:52:26--  http://alfonseca.org/pubs/ws353simrel.tar.gz
Resolving alfonseca.org (alfonseca.org)... 162.215.249.67
Connecting to alfonseca.org (alfonseca.org)|162.215.249.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5460 (5.3K) [application/x-gzip]
Saving to: ‘ws353simrel.tar.gz’

ws353simrel.tar.gz  100%[===================>]   5.33K  --.-KB/s    in 0s      

2023-06-11 17:52:26 (628 MB/s) - ‘ws353simrel.tar.gz’ saved [5460/5460]

wordsim353_sim_rel/wordsim353_agreed.txt
wordsim353_sim_rel/wordsim353_annotator1.txt
wordsim353_sim_rel/wordsim353_annotator2.txt
wordsim353_sim_rel/wordsim_relatedness_goldstandard.txt
wordsim353_sim_rel/wordsim_similarity_goldstandard.txt
tiger	cat	7.35
tiger	tiger	10.00
plane	car	5.77
train	car	6.31
television	radio	6.77


In [ ]:
import pandas as pd

df = pd.read_csv("wordsim353_sim_rel/wordsim_similarity_goldstandard.txt",
                 sep="\t", header=None)
df.columns = ["first", "second", "score"]
df.head(3)

,first,second,score
0,tiger,cat,7.35
1,tiger,tiger,10.00
2,plane,car,5.77


In [ ]:
# Load the gold standard similarity scores
gold_df = pd.read_csv("wordsim353_sim_rel/wordsim_similarity_goldstandard.txt", sep="\t", header=None)
gold_df.columns = ["first", "second", "gold_score"]

# Merge the gold standard dataframe with your own scores dataframe
merged_df = pd.merge(gold_df, my_scores_df, on=["first", "second"])

# Calculate the Spearman correlation coefficient between the gold standard scores and your own scores
corr, p_value = spearmanr(merged_df["gold_score"], merged_df["score"])
print("Spearman correlation coefficient:", corr)

NameError: ignored

In [ ]:
from scipy.stats import spearmanr

# Merge with human similarity ratings on word pairs
merged_df = pd.merge(df, model_preds, on=["first", "second"])

# Calculate the Spearman correlation coefficient and p-value
corr, pval = spearmanr(merged_df["score_x"], merged_df["score_y"])

Можно заметить, что модель google-news-vectors несколько выигрывает в данном случае.

# Задание начинается тут

In [ ]:
#1. Определите косинусное расстояние между векторами слов: media и bread
print("Косинусное расстояние GN", round(w.distance("media", "bread"), 3))
print("Косинусное расстояние BR", round(w_british.distance("media_NOUN", "bread_NOUN"), 3))

Косинусное расстояние GN 0.839
Косинусное расстояние BR 0.886


In [ ]:
#2. Дан набор слов: media bread cucumber doctor, определите лишнее слово.
gn_words = "media bread cucumber doctor".split()
br_words = list(map(lambda x: x + "_NOUN", gn_words))
print(br_words)
print("Лишнее слово GN", w.doesnt_match(gn_words))
print("Лишнее слово BR (без _NOUN)", w_british.doesnt_match(br_words))

['media_NOUN', 'bread_NOUN', 'cucumber_NOUN', 'doctor_NOUN']
Лишнее слово GN media
Лишнее слово BR (без _NOUN) media_NOUN


In [ ]:
#3. Определите косинусное расстояние между векторами предложений:
# chain is only as strong as its weakest link
# Actions speak louder than words

import numpy as np

#enter your code here
# предложение
sentence_1 = 'chain is only as strong as its weakest link'
sentence_2 = 'Actions speak louder than words'

# список слов в предложении
words_1 = sentence_1.split()
words_2 = sentence_2.split()

# получение вектора каждого слова
word_vectors_1 = [w[word] for word in words_1]
word_vectors_2 = [w[word] for word in words_2]

# усреднение векторов слов
sentence_vector_1 = np.mean(word_vectors_1, axis=0)
sentence_vector_2 = np.mean(word_vectors_2, axis=0)

from scipy.spatial.distance import cosine
print("distance", round(cosine(sentence_vector_1, sentence_vector_2), 3))

distance 0.784


In [ ]:
#4. Из набора данных word_sim извлеките подвыборку пар слов с индексами 14:114 (нумерация начинается с нуля, правая граница не включается).

#Из уже выбранных пар используйте только те, для слов которых находятся векторы на британском корпусе, помеченные как существительные! Иначе удаляйте такую пару из подвыборки.

#Вычислите корреляцию Спирмена между оценками схожести выбранных пар слов, полученных в результате работы моделей, и оценками аннотаторов в датасете word_sim.

import pandas as pd
from scipy.stats import spearmanr

df = pd.read_csv("wordsim353_sim_rel/wordsim_similarity_goldstandard.txt",
                 sep="\t", header=None)
df.columns = ["first", "second", "score"]
df.head(3)

delete_count = 0

gn_dist, br_dist, scores = [], [], []

# 4. Из набора данных word_sim извлеките подвыборку пар слов с индексами 14:114 (нумерация начинается с нуля, правая граница не включается).
for row in df.iloc[14:114].iterrows():
    w1, w2 = row[1]["first"], row[1]["second"]
    try:
        #enter your code here

        bk1 = w1.lower() + "_NOUN"
        bk2 = w2.lower() + "_NOUN"

        br_dist.append(w_british.similarity(bk1, bk2))

        gn_dist.append(w.similarity(w1, w2))

        scores.append(row[1]["score"])

    except KeyError as e:
        delete_count += 1
        print(e, "Skipping this word.")

print("GN спирман", spearmanr(gn_dist, scores))
print("BR спирман", spearmanr(br_dist, scores))
print("количество удаленных слов", delete_count)

"Key 'arafat_NOUN' not present" Skipping this word.
"Key 'harvard_NOUN' not present" Skipping this word.
"Key 'mexico_NOUN' not present" Skipping this word.
GN спирман SignificanceResult(statistic=0.6938116658940471, pvalue=3.3231348509522888e-15)
BR спирман SignificanceResult(statistic=0.6460421691947137, pvalue=8.94897260473219e-13)
количество удаленных слов 3


In [ ]:
round(0.6938116658940471, 3)

0.694

In [ ]:
round(0.6460421691947137, 3)

0.646

In [ ]:
#4. Из набора данных word_sim извлеките подвыборку пар слов с индексами 26:126 (нумерация начинается с нуля, правая граница не включается).

#Из уже выбранных пар используйте только те, для слов которых находятся векторы на британском корпусе, помеченные как существительные! Иначе удаляйте такую пару из подвыборки.

#Вычислите корреляцию Спирмена между оценками схожести выбранных пар слов, полученных в результате работы моделей, и оценками аннотаторов в датасете word_sim.

import pandas as pd

df = pd.read_csv("wordsim353_sim_rel/wordsim_similarity_goldstandard.txt",
                 sep="\t", header=None)
df.columns = ["first", "second", "score"]
df.head(3)

delete_count = 0

gn_dist, br_dist, scores = [], [], []

for row in df.iloc[26:126].iterrows():
    w1, w2 = row[1]["first"], row[1]["second"]
    try:
        #enter your code here

        bk1 = w1.lower() + "_NOUN"
        bk2 = w2.lower() + "_NOUN"

        br_dist.append(w_british.similarity(bk1, bk2))

        gn_dist.append(w.similarity(w1, w2))

        scores.append(row[1]["score"])

    except KeyError as e:
        delete_count += 1
        print(e, "Skipping this word.")

print("GN спирман", spearmanr(gn_dist, scores))
print("BR спирман", spearmanr(br_dist, scores))
print("количество удаленных слов", delete_count)

"Key 'harvard_NOUN' not present" Skipping this word.
"Key 'mexico_NOUN' not present" Skipping this word.
GN спирман SignificanceResult(statistic=0.7052214812926481, pvalue=5.227025361914796e-16)
BR спирман SignificanceResult(statistic=0.6800673363070284, pvalue=1.3508448092503549e-14)
количество удаленных слов 2
